In [10]:
# Tweet Topic Modeling with BERTopic
# ============================

# This notebook performs topic modeling on a large database of tweets using BERTopic.
# Since the database is large, we use efficient loading techniques and batch processing.
# We use pysentimiento for specialized tweet preprocessing before applying BERTopic.

import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
from tqdm.notebook import tqdm
import re
import nltk
from nltk.corpus import stopwords
from pysentimiento.preprocessing import preprocess_tweet
import warnings
warnings.filterwarnings("ignore")

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

In [11]:
# Database connection function
def connect_to_db(db_path):
    """Connect to the SQLite database."""
    try:
        conn = sqlite3.connect(db_path)
        print(f"Successfully connected to database: {db_path}")
        return conn
    except sqlite3.Error as e:
        print(f"Error connecting to database: {e}")
        return None

# 1. Initial Database Exploration
# ===============================

# Connect to the database
db_path = "english_tweets_only.db"
conn = connect_to_db(db_path)


In [17]:
# Function to get table info
def get_table_info(conn, table_name):
    """Get information about the specified table."""
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    print(f"\nTable: {table_name}")
    print("=" * (len(table_name) + 7))
    for col in columns:
        print(f"{col[1]} ({col[2]})")

# Get table information
get_table_info(conn, "posts")
get_table_info(conn, "authors")

In [15]:
# Function to get table statistics
def get_table_stats(conn, table_name):
    """Get basic statistics about the table."""
    cursor = conn.cursor()
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"\nTable '{table_name}' contains {count:,} records")
    
    if table_name == "posts":
        # Get date range
        cursor.execute("SELECT MIN(date), MAX(date) FROM posts")
        min_date, max_date = cursor.fetchone()
        print(f"Date range: {min_date} to {max_date}")
        
        # Get top 5 authors by post count
        cursor.execute("""
            SELECT author, COUNT(*) as post_count 
            FROM posts 
            GROUP BY author 
            ORDER BY post_count DESC 
            LIMIT 5
        """)
        print("\nTop 5 authors by post count:")
        for author, count in cursor.fetchall():
            print(f"  {author}: {count:,} posts")

# Get table statistics
get_table_stats(conn, "posts")
get_table_stats(conn, "authors")

In [ ]:
# 2. Data Preprocessing
# ====================

def preprocess_tweets(text):
    """Clean and preprocess tweet text using pysentimiento and additional cleaning."""
    if not isinstance(text, str):
        return ""
    
    # Use pysentimiento's specialized tweet preprocessor first
    # This handles mentions, URLs, emojis, and other Twitter-specific elements
    processed_text = preprocess_tweet(text)
    
    # Additional cleaning
    # Remove any remaining special characters and numbers
    processed_text = re.sub(r'[^\w\s]', '', processed_text)
    processed_text = re.sub(r'\d+', '', processed_text)
    
    # Convert to lowercase
    processed_text = processed_text.lower()
    
    # Remove extra whitespace
    processed_text = re.sub(r'\s+', ' ', processed_text).strip()
    
    return processed_text


2025-04-14 14:35:19,327 - __main__ - INFO - Successfully connected to database at 53k_individual_hcps_70_percent_confidence_tweets_2019_2022.db
2025-04-14 14:35:21,996 - __main__ - WARNING - No tweets found with language='en'. Fetching all tweets to analyze language...
2025-04-14 14:38:14,363 - __main__ - INFO - Fetched 16616970 total tweets for language detection
2025-04-14 14:38:21,827 - __main__ - INFO - Detecting English tweets from mixed language dataset...


In [ ]:
# 3. Efficient Data Loading
# ========================

def load_tweets_in_batches(conn, batch_size=10000, max_tweets=None):
    """
    Load tweets in batches to handle large datasets efficiently.
    
    Args:
        conn: Database connection
        batch_size: Number of tweets to load in each batch
        max_tweets: Maximum number of tweets to load (None for all)
        
    Returns:
        List of preprocessed tweet texts
    """
    cursor = conn.cursor()
    
    # Get total count if needed
    if max_tweets is None:
        cursor.execute("SELECT COUNT(*) FROM posts")
        max_tweets = cursor.fetchone()[0]
    
    # Initialize variables
    all_tweets = []
    offset = 0
    total_loaded = 0
    
    print(f"Loading up to {max_tweets:,} tweets in batches of {batch_size:,}")
    
    # Load in batches
    pbar = tqdm(total=min(max_tweets, max_tweets))
    while total_loaded < max_tweets:
        # Adjust batch size for last batch if needed
        current_batch_size = min(batch_size, max_tweets - total_loaded)
        
        # Execute query for current batch
        cursor.execute(f"""
            SELECT content 
            FROM posts 
            LIMIT {current_batch_size} OFFSET {offset}
        """)
        
        # Process batch
        batch_tweets = [preprocess_tweets(row[0]) for row in cursor.fetchall()]
        batch_tweets = [tweet for tweet in batch_tweets if len(tweet) > 20]  # Filter very short tweets
        
        # Add to collection
        all_tweets.extend(batch_tweets)
        
        # Update counters
        batch_actual_size = len(batch_tweets)
        total_loaded += batch_actual_size
        offset += current_batch_size
        pbar.update(batch_actual_size)
        
        # Break if no more tweets
        if batch_actual_size == 0:
            break
    
    pbar.close()
    print(f"Loaded {len(all_tweets):,} tweets after preprocessing and filtering")
    return all_tweets

# Load tweets (adjust max_tweets as needed for initial testing)
# For initial testing, you might want to use a smaller sample
tweets = load_tweets_in_batches(conn, batch_size=10000, max_tweets=100000)  # Adjust max_tweets as needed


In [ ]:
# 4. Topic Modeling with BERTopic
# ==============================

def create_topic_model(tweets, nr_topics="auto"):
    """
    Create and train a BERTopic model.
    
    Args:
        tweets: List of preprocessed tweet texts
        nr_topics: Number of topics to extract ("auto" to determine automatically)
        
    Returns:
        Trained BERTopic model
    """
    print("Setting up the topic modeling pipeline...")
    
    # Create sentence transformer model for embeddings
    embedding_model = SentenceTransformer("all-MiniLM-L6-v2")  # Smaller model for efficiency
    
    # Set up dimensionality reduction
    umap_model = UMAP(n_neighbors=15, 
                      n_components=5,
                      min_dist=0.0, 
                      metric='cosine', 
                      random_state=42)
    
    # Set up clustering model
    hdbscan_model = hdbscan.HDBSCAN(min_cluster_size=10,
                                   min_samples=5,
                                   metric='euclidean',
                                   cluster_selection_method='eom',
                                   prediction_data=True)
    
    # Set up custom vectorizer with additional stopwords
    stop_words = list(stopwords.words('english'))
    additional_stopwords = ['rt', 'RT', 'amp', 'twitter', 'tweet', 'tweeting']
    stop_words.extend(additional_stopwords)
    
    vectorizer_model = CountVectorizer(stop_words=stop_words)
    
    # Create the BERTopic model
    topic_model = BERTopic(
        embedding_model=embedding_model,
        umap_model=umap_model,
        hdbscan_model=hdbscan_model,
        vectorizer_model=vectorizer_model,
        nr_topics=nr_topics,
        calculate_probabilities=False,  # Turn off for large datasets to save memory
        verbose=True
    )
    
    print("Training BERTopic model...")
    topics, probs = topic_model.fit_transform(tweets)
    
    print(f"Model training complete. Found {len(topic_model.get_topic_info())-1} topics.")
    return topic_model, topics

# Create and train the model
# This may take time depending on your dataset size and available compute resources
topic_model, topics = create_topic_model(tweets, nr_topics="auto")

In [ ]:
# 5. Analyzing Topics
# =================

# Get topic information
topic_info = topic_model.get_topic_info()
print("\nTopic Distribution:")
print(topic_info.head(10))

# Visualize topic size distribution
plt.figure(figsize=(12, 6))
sizes = topic_info[topic_info['Topic'] != -1]['Count'].values
plt.hist(sizes, bins=30)
plt.xlabel('Topic Size (Number of Tweets)')
plt.ylabel('Number of Topics')
plt.title('Distribution of Topic Sizes')
plt.tight_layout()
plt.show()

# Get the top topics (excluding the -1 outlier topic)
top_topics = topic_info[topic_info['Topic'] != -1].head(10)['Topic'].values

# Print the top terms for top topics
print("\nTop Terms for Major Topics:")
for topic in top_topics:
    print(f"\nTopic {topic}:")
    for term, weight in topic_model.get_topic(topic)[:10]:
        print(f"  {term}: {weight:.4f}")


In [ ]:
# 6. Visualizations
# ================

# Topic similarity map
print("\nGenerating topic similarity visualization...")
try:
    fig = topic_model.visualize_topics()
    fig.show()
except Exception as e:
    print(f"Could not generate topic similarity visualization: {e}")

# Topic word scores
print("\nGenerating word score visualizations for top topics...")
for topic in top_topics[:5]:  # Show for first 5 top topics
    try:
        fig = topic_model.visualize_barchart(topics=[topic], top_n_topics=1)
        fig.show()
    except Exception as e:
        print(f"Could not generate word score visualization for topic {topic}: {e}")

# Topic hierarchy
print("\nGenerating topic hierarchy visualization...")
try:
    fig = topic_model.visualize_hierarchy()
    fig.show()
except Exception as e:
    print(f"Could not generate topic hierarchy visualization: {e}")


In [ ]:
# 7. Dynamic Topic Modeling (Optional)
# ==================================

def perform_dynamic_topic_modeling(conn, topic_model, time_periods=5):
    """
    Analyze how topics evolve over time.
    
    Args:
        conn: Database connection
        topic_model: Trained BERTopic model
        time_periods: Number of time periods to divide the data into
    """
    print("\nPerforming dynamic topic modeling...")
    
    # Get date range
    cursor = conn.cursor()
    cursor.execute("SELECT MIN(date), MAX(date) FROM posts")
    min_date, max_date = cursor.fetchone()
    
    try:
        # Convert to datetime if possible
        min_date = pd.to_datetime(min_date)
        max_date = pd.to_datetime(max_date)
        
        # Create time bins
        date_bins = pd.date_range(start=min_date, end=max_date, periods=time_periods+1)
        date_labels = [f"Period {i+1}" for i in range(time_periods)]
        
        # Fetch tweets with dates
        cursor.execute("SELECT content, date FROM posts LIMIT 50000")  # Limit for memory considerations
        df = pd.DataFrame(cursor.fetchall(), columns=['content', 'date'])
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df.dropna(subset=['date'])
        
        # Assign time periods
        df['time_period'] = pd.cut(df['date'], bins=date_bins, labels=date_labels, include_lowest=True)
        
        # Preprocess tweets
        df['processed_content'] = df['content'].apply(preprocess_tweets)
        
        # Filter out short tweets
        df = df[df['processed_content'].str.len() > 20]
        
        # Get documents and timestamps
        documents = df['processed_content'].tolist()
        timestamps = df['time_period'].tolist()
        
        # Run dynamic topic modeling
        topics_over_time = topic_model.topics_over_time(documents, timestamps)
        
        # Visualize
        fig = topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)
        fig.show()
        
    except Exception as e:
        print(f"Error in dynamic topic modeling: {e}")
        print("Skipping dynamic topic analysis.")

# Optional: Uncomment to run dynamic topic modeling
# perform_dynamic_topic_modeling(conn, topic_model)

In [ ]:
# 8. Export Results
# ===============

# Save model
topic_model.save("tweet_bertopic_model")

# Export topic information to CSV
topic_info.to_csv("tweet_topics_info.csv", index=False)

# Export top terms for all topics
all_topics_terms = {}
for topic in topic_info['Topic'].values:
    if topic != -1:  # Skip outlier topic
        all_topics_terms[topic] = topic_model.get_topic(topic)

# Convert to DataFrame and save
topics_df = pd.DataFrame({
    'Topic': [topic for topic in all_topics_terms.keys() for _ in range(10)],
    'Term': [term for terms in all_topics_terms.values() for term, _ in terms[:10]],
    'Weight': [weight for terms in all_topics_terms.values() for _, weight in terms[:10]]
})
topics_df.to_csv("tweet_topic_terms.csv", index=False)


In [ ]:
# 9. Assign Topics to All Tweets
# ============================

def assign_topics_to_tweets(conn, topic_model, batch_size=10000, max_tweets=None):
    """
    Assign topics to all tweets in the database.
    
    Args:
        conn: Database connection
        topic_model: Trained BERTopic model
        batch_size: Number of tweets to process in each batch
        max_tweets: Maximum number of tweets to process (None for all)
    """
    cursor = conn.cursor()
    
    # Get total count if needed
    if max_tweets is None:
        cursor.execute("SELECT COUNT(*) FROM posts")
        max_tweets = cursor.fetchone()[0]
    
    # Initialize variables
    offset = 0
    total_processed = 0
    results = []
    
    print(f"Assigning topics to up to {max_tweets:,} tweets in batches of {batch_size:,}")
    
    # Process in batches
    pbar = tqdm(total=min(max_tweets, max_tweets))
    while total_processed < max_tweets:
        # Adjust batch size for last batch if needed
        current_batch_size = min(batch_size, max_tweets - total_processed)
        
        # Execute query for current batch
        cursor.execute(f"""
            SELECT post_id, content 
            FROM posts 
            LIMIT {current_batch_size} OFFSET {offset}
        """)
        
        # Get batch data
        batch_data = cursor.fetchall()
        if not batch_data:
            break
            
        batch_ids = [row[0] for row in batch_data]
        batch_tweets = [preprocess_tweets(row[1]) for row in batch_data]
        
        # Predict topics
        try:
            batch_topics, _ = topic_model.transform(batch_tweets)
            
            # Collect results
            for tweet_id, topic in zip(batch_ids, batch_topics):
                results.append((tweet_id, topic))
            
        except Exception as e:
            print(f"Error processing batch at offset {offset}: {e}")
            
        # Update counters
        batch_actual_size = len(batch_data)
        total_processed += batch_actual_size
        offset += current_batch_size
        pbar.update(batch_actual_size)
    
    pbar.close()
    print(f"Assigned topics to {len(results):,} tweets")
    
    # Convert to DataFrame and save
    topics_df = pd.DataFrame(results, columns=['post_id', 'topic'])
    topics_df.to_csv("tweet_topic_assignments.csv", index=False)
    
    return topics_df

# Optional: Uncomment to assign topics to all tweets
# tweet_topics = assign_topics_to_tweets(conn, topic_model, batch_size=10000, max_tweets=None)


In [ ]:
# 10. Topic Analysis by Author Metrics
# =================================

def analyze_topics_by_author_metrics(conn, tweet_topics):
    """
    Analyze how topics correlate with author metrics.
    
    Args:
        conn: Database connection
        tweet_topics: DataFrame with post_id and topic
    """
    print("\nAnalyzing topics by author metrics...")
    
    try:
        # Join posts with authors and topics
        query = """
        SELECT 
            t.topic,
            a.followers_count,
            a.statuses_count,
            p.retweet_count,
            p.like_count,
            p.reply_count
        FROM 
            posts p
        JOIN 
            authors a ON p.author_osn_id = a.author_osn_id
        JOIN 
            tweet_topic_assignments t ON p.post_id = t.post_id
        WHERE
            t.topic != -1
        LIMIT 100000
        """
        
        # Create temporary table for topic assignments
        cursor = conn.cursor()
        cursor.execute("DROP TABLE IF EXISTS tweet_topic_assignments")
        cursor.execute("CREATE TEMPORARY TABLE tweet_topic_assignments (post_id REAL, topic INTEGER)")
        
        # Insert topic assignments in batches
        chunk_size = 10000
        for i in range(0, len(tweet_topics), chunk_size):
            chunk = tweet_topics.iloc[i:i+chunk_size]
            chunk_data = [(row.post_id, row.topic) for _, row in chunk.iterrows()]
            cursor.executemany("INSERT INTO tweet_topic_assignments VALUES (?, ?)", chunk_data)
        
        # Run the query
        df = pd.read_sql_query(query, conn)
        
        # Calculate metrics by topic
        topic_metrics = df.groupby('topic').agg({
            'followers_count': 'mean',
            'statuses_count': 'mean',
            'retweet_count': 'mean',
            'like_count': 'mean',
            'reply_count': 'mean'
        }).reset_index()
        
        # Save results
        topic_metrics.to_csv("topic_author_metrics.csv", index=False)
        
        # Visualize relationship between topics and engagement
        plt.figure(figsize=(12, 8))
        
        # Sort topics by engagement (retweet + like + reply counts)
        topic_metrics['total_engagement'] = topic_metrics['retweet_count'] + topic_metrics['like_count'] + topic_metrics['reply_count']
        topic_metrics = topic_metrics.sort_values('total_engagement', ascending=False)
        
        # Plot top 20 topics by engagement
        top_n = min(20, len(topic_metrics))
        topics = topic_metrics['topic'].values[:top_n]
        
        plt.bar(range(top_n), topic_metrics['total_engagement'].values[:top_n])
        plt.xticks(range(top_n), topics, rotation=90)
        plt.xlabel('Topic')
        plt.ylabel('Average Engagement (RT + Like + Reply)')
        plt.title('Topics by Average Engagement')
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Error in topic-author analysis: {e}")

# Optional: Uncomment to analyze topics by author metrics
# analyze_topics_by_author_metrics(conn, tweet_topics)

In [ ]:
# 11. Close Connection
# ==================

# Close the database connection
conn.close()
print("Database connection closed.")
print("\nAnalysis complete! Results saved to CSV files.")